In [ ]:
import os
import sys

sys.path.append(os.path.abspath(".."))

import polars as pl
from pathlib import Path

from src.utils.utils import has_converged_to_price
from src.plotting.final_figures import plot_monopoly_experiment_svg
from src.utils.experiments_export import concat_experiments_to_parquet

OUPUT_PATH = Path("../latex/imgs/results/")
OUPUT_PATH.mkdir(parents=True, exist_ok=True)
DATA_OUTPUT_PATH = Path("../data/results/")
DATA_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
INPUT_PATH = Path("../experiments_synthetic/experiments_runs/")

# Functions
---

## Monopoly


In [ ]:
concat_experiments_to_parquet(
    experiment_dirs=INPUT_PATH,
    output_path=DATA_OUTPUT_PATH / "all_experiments.parquet",
    rebalance_exper=True,
    min_rebalance=7,
    keep_min_exactly=False,
)

Unfinished experiments found (SKIPPED):
 - 3_agents/1751112912_oligopoly_setting_3_firms_P2
 - 2_agents/1751225659_duopoly_setting_P2_asymmetric
 - 3_agents/1751315360_oligopoly_setting_3_firms_P2
 - 2_agents/1751050177_duopoly_setting_P1
 - 2_agents/1751052605_duopoly_setting_P1
 - 3_agents/1751117060_oligopoly_setting_3_firms_P2
 - 3_agents/1751106785_oligopoly_setting_3_firms_P2
 - 3_agents/1751121653_oligopoly_setting_3_firms_P2
 - 3_agents/1751117083_oligopoly_setting_3_firms_P2
-----> Missing experiment runs on: is_synthetic=True, is_symmetric=False, num_agents=2 for prefix_type=P2 and alpha=10 (only 6 runs)

Total incomplete groups: 1
Rebalancing will be done only on complete groups.
Final runs count by (is_synthetic, is_symmetric, num_agents, alpha, agent_prefix_type):
is_synthetic=True, is_symmetric=True, num_agents=1:
  alpha=1.0, prefix=OTHER => runs: 2
  alpha=3.2, prefix=OTHER => runs: 2
  alpha=10.0, prefix=OTHER => runs: 2

is_synthetic=True, is_symmetric=True, num_agent

# Plots
---

## Monopoly

In [ ]:
MONOPOLY_OUPUT_PATH = Path(OUPUT_PATH) / "monopoly"
MONOPOLY_INPUT_PATH = Path(INPUT_PATH) / "1_agents"
MONOPOLY_OUPUT_PATH.mkdir(parents=True, exist_ok=True)

In [ ]:
for folder in MONOPOLY_INPUT_PATH.iterdir():
    if not folder.is_dir():
        continue
    print(f"Processing folder: {folder.name}")
    metadata = pl.read_json(folder / "metadata.json").to_dicts()[0]
    df = pl.read_parquet(folder / "environment_history.parquet")

    price_monopoly = (
        metadata.get("environment").get("environment_params").get("monopoly_prices")[0]
    )
    price_series = df.select("price").to_series().to_list()
    has_converged_to_price(price_series, price_monopoly)

In [ ]:
df = pl.read_parquet(
    MONOPOLY_INPUT_PATH / "1750960646_monopoly_setting" / "environment_history.parquet"
)

In [ ]:
plot_monopoly_experiment_svg(
    df=df,
    title="Monopoly Experiment",
    metadata=metadata,
    save_path=MONOPOLY_OUPUT_PATH / "monopoly_experiment_complete",
    show_quantities=False,
    show_profits=False,
    plot_references=True,
    display=False,
)

In [ ]:
results = []

for folder in MONOPOLY_INPUT_PATH.iterdir():
    if not folder.is_dir():
        continue

    metadata = pl.read_json(folder / "metadata.json").to_dicts()[0]
    df = pl.read_parquet(folder / "environment_history.parquet")

    alpha = metadata["environment"]["environment_params"]["alpha"][0]

    # Default model name
    model_name = "mistral-large-2411"
    agents_models = metadata.get("agents_models")
    if agents_models is not None and "Firm A" in agents_models:
        model_name = agents_models["Firm A"]

    price_monopoly = (
        metadata["environment"]["environment_params"]["monopoly_prices"][0] / alpha
    )
    price_series = (df.select("price").to_series().to_numpy() / alpha).tolist()

    run_name = folder.name.split("_")[0]

    model_size = "Small" if "small" in model_name.lower() else "Large"

    # Create one row per price
    for round_num, price in enumerate(price_series, start=1):
        results.append(
            {
                # "model_name": model_name,
                # "alpha": alpha,
                "agent": f"{model_size}",  # -alpha={alpha}",
                "agent_type": "LLM",
                "price": price,
                "monopoly_price": price_monopoly,
                "deviated": price_monopoly - price,
                "dev_percent": (price_monopoly - price) / price_monopoly * 100,
                "outside_range": int(
                    price < price_monopoly * 0.95 or price > price_monopoly * 1.05
                ),
                "run": run_name,
                "round": round_num,
            }
        )

df = pl.DataFrame(results).filter(pl.col("round") > 200).sort("round")
df

In [ ]:
# run statistics by agent type and price, deviation and dev_percent
stats = (
    df.group_by(["agent"])
    .agg(
        [
            pl.mean("price").alias("mean_price"),
            pl.mean("deviated").alias("mean_deviated"),
            pl.mean("dev_percent").alias("mean_dev_percent"),
            pl.std("price").alias("std_price"),
            pl.std("dev_percent").alias("std_dev_percent"),
            pl.sum("outside_range").alias("outside_range_count"),
        ]
    )
    .sort("agent")
)
stats = stats.to_pandas()
stats